In [1]:
#all imports
import tensorflow as tf
import torch

In [2]:

gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  raise(SystemError('GPU device not found'))

Found GPU at :/device:GPU:0


In [3]:
if torch.cuda.is_available():
  device=torch.device("cuda")
  print("There are %d GPU DEVICES available " %torch.cuda.device_count())
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")



There are 1 GPU DEVICES available 
The device name is Tesla K80


In [4]:
!pip install transformers

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Custom Data set and Data loader has been inspired from 
#https://github.com/sugi-chan/custom_bert_pipeline/blob/master/bert_pipeline.ipynb

In [6]:
!unzip -P ****** -qq '/content/drive/My Drive/TurkishData/Turkish1.zip'


replace Turkish/readme-trainingset-tr.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Turkish/offenseval-annotation.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Turkish/offenseval-tr-training-v1.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [7]:
print("Hello")

Hello


In [0]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a']
greekdata = pd.read_csv("Turkish/offenseval-tr-training-v1.tsv", delimiter='\t',names=headers)
data=greekdata[1:]


In [10]:
data.head()

,id,tweet,subtask_a
1,20948,@USER en güzel uyuyan insan ödülü jeon jungkoo...,NOT
2,10134,"@USER Mekanı cennet olsun, saygılar sayın avuk...",NOT
3,23457,Kızlar aranızda kas yığını beylere düşenler ol...,NOT
4,18401,Biraz ders çalışayım. Tembellik ve uyku düşman...,NOT
5,17525,@USER Trezeguet yerine El Sharawy daha iyi olm...,NOT


In [11]:
len(data)

31277

In [0]:

dfnumpy=data.to_numpy();
x=dfnumpy[:, 1].reshape(-1, 1)
y=dfnumpy[:, 2].reshape(-1, 1)


STop words taken from :https://pypi.org/project/stop-words/

In [13]:
pip install emoji --upgrade

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.5.4)


In [0]:
import re

def ft1(l):
    a = re.split('|\]|\[|\)|\(|; |,|\*|\n',l)
    li = []
    for t in a:
        li.append(t.strip(' ,')) 
    return li

In [0]:
#Removing duplicate words inspired from https://stackoverflow.com/questions/57424661/how-to-efficiently-remove-consecutive-duplicate-words-or-phrases-in-a-string

In [0]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
import re
import emoji
import string
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
def preprocessinglib1(arrt):
    ans =[]
    for txt in arrt:
        txt1=emoji.demojize(txt)
        unique_words = dict.fromkeys(txt1.split())
        txt2=' '.join(unique_words)
        tokens=tknzr.tokenize(txt2)
        ctokens=[]
        for t in tokens:
          if t not in string.punctuation :
            ctokens.append(t)
        clean_s = ' '.join(ctokens)
        ans.append(clean_s)
               
    return ans

In [0]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
import emoji
import string
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
def preprocessinglib(arrt):
    ans =[]
    for txt in arrt:
        txt1=emoji.demojize(txt)
        unique_words = dict.fromkeys(txt1.split(" "))
        txt2=' '.join(unique_words)
        ans.append(ft1(txt2))
               
    return ans

In [18]:
'''import json
x=""
y=""

with open('Danish/tweet.json','r') as xjson:
    x=json.load(xjson)
x=x[1:]
print(x[:5])
with open('Danish/label.json','r') as yjson:
    y=json.load(yjson)

y=y[1:]
print(y[:5])'''

'import json\nx=""\ny=""\n\nwith open(\'Danish/tweet.json\',\'r\') as xjson:\n    x=json.load(xjson)\nx=x[1:]\nprint(x[:5])\nwith open(\'Danish/label.json\',\'r\') as yjson:\n    y=json.load(yjson)\n\ny=y[1:]\nprint(y[:5])'

In [19]:
arrt=x[:,0]
allTokens=preprocessinglib(arrt)
#allTokenstrain, allTokenstest, y_train, y_test = train_test_split(allTokens,y, test_size=0.2, random_state=42)
#allTokensPredict, _, y_predict, _ = train_test_split(allTokenstest,y_test, test_size=0.5, random_state=42)


/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [0]:
preprocessedTweets=allTokens

In [21]:
preprocessedTweets[:5]

[["@USER en güzel uyuyan insan ödülü jeon jungkook'a gidiyor..."],
 ['@USER Mekanı cennet olsun, saygılar sayın avukatımız,iyi günler dilerim'],
 ['Kızlar aranızda kas yığını beylere düşenler olduğunu görüyorum yapmayın allah aşkına'],
 ['Biraz ders çalışayım. Tembellik ve uyku düşmanımızdır.'],
 ['@USER Trezeguet yerine El Sharawy daha iyi olmaz mı']]

In [0]:

from sklearn.utils import shuffle
preprocessedTweets, y = shuffle(preprocessedTweets, y)

In [23]:
y

array([['NOT'],
       ['NOT'],
       ['NOT'],
       ...,
       ['OFF'],
       ['NOT'],
       ['NOT']], dtype=object)

In [0]:
with open('/content/drive/My Drive/preprocessedTweetsTurkish.txt', 'w') as f:
  f.write(str(x))


In [25]:
print(preprocessedTweets[0:5])

[['Artık hiçbir şeye o kadar tahammülüm kalmadı ki şarkının introsu 20 saniyeyse 20.saniyeye almadan dinleyemiyorum.'], ['@USER Dedi ve tezini yazmaya devam etti.'], ['@USER Of gerçekten bu kalçalar harika'], ['@USER senaristler genç olunca işte hevesli oluyorlar manifesto çekmeye'], ['beş bin yüz on altı  #MTVHottest Demi Lovato']]


In [26]:
from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

Using TensorFlow backend.


In [0]:
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)


In [0]:
yavg=[]

In [29]:
'''ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]
MAXLENGTH=""
def giveIds(sentence,y_):
  ids_of_sentence=[]
  ids_of_sentence_words=[]
  attention_masks=[]
  maxlength=0
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t[0],add_special_tokens=True)
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
      ids_of_sentence.append(tokenized_sentence_id)
      yavg.append(len(tokenized_sentence_id))
  MAXLENGTH=maxlength
  return MAXLENGTH,yavg
  #ids_of_sentence_words=pad_sequences(ids_of_sentence,maxlen=maxlength,dtype="long",value=0,truncating="post",padding="post")##can change max length
  #attention_masks = [[int(a > 0)   for a in b ]for b in ids_of_sentence_words] 
  #print(len(attention_masks))
  #print(len(ids_of_sentence_words))
  #return ids_of_sentence_words,attention_masks'''

'ids_of_sentence=[]\nids_of_sentence_words=[]\nattention_masks=[]\nMAXLENGTH=""\ndef giveIds(sentence,y_):\n  ids_of_sentence=[]\n  ids_of_sentence_words=[]\n  attention_masks=[]\n  maxlength=0\n  for t in sentence:\n      tokenized_sentence_id=tokenizer.encode(t[0],add_special_tokens=True)\n      if(maxlength<len(tokenized_sentence_id)):\n          maxlength=len(tokenized_sentence_id)\n      ids_of_sentence.append(tokenized_sentence_id)\n      yavg.append(len(tokenized_sentence_id))\n  MAXLENGTH=maxlength\n  return MAXLENGTH,yavg\n  #ids_of_sentence_words=pad_sequences(ids_of_sentence,maxlen=maxlength,dtype="long",value=0,truncating="post",padding="post")##can change max length\n  #attention_masks = [[int(a > 0)   for a in b ]for b in ids_of_sentence_words] \n  #print(len(attention_masks))\n  #print(len(ids_of_sentence_words))\n  #return ids_of_sentence_words,attention_masks'

In [0]:
#print(tokenizer.tokenize(preprocessedTweets[0][0]))

In [31]:
#sentence_train, sentence_test, y_train, y_test= train_test_split(preprocessedTweets,y, test_size=0.1, random_state=42)
#x_train_mask,x_test_mask,_,_=train_test_split(attention_masks,y, test_size=0.2, random_state=42)
'''MAXLENGTH,yavg=giveIds(preprocessedTweets,y)
print("Average length is ",sum(yavg)/len(yavg))'''

'MAXLENGTH,yavg=giveIds(preprocessedTweets,y)\nprint("Average length is ",sum(yavg)/len(yavg))'

In [0]:
#MAXLENGTH is 6151 average lentgh is 

In [0]:
sentence_train, sentence_test, y_train, y_test = train_test_split(preprocessedTweets,y, test_size=0.2, random_state=42)
#x_train_mask,x_test_mask,_,_=train_test_split(attention_masks,y, test_size=0.2, random_state=42)



In [0]:
##do this at the time of training only 
#sentence_train, sentence_test1, y_train, y_test1 = train_test_split(preprocessedTweets,y, test_size=0.2, random_state=42) only this for code submission
#sentence_test,sentence_predict, y_test,y_predict = train_test_split(sentence_test1,y_test1, test_size=0.5, random_state=42)
#x_predict_mask,_,_,_=train_test_split(x_test_mask,y_test, test_size=0.5, random_state=42)

In [35]:
y_test

array([['NOT'],
       ['NOT'],
       ['NOT'],
       ...,
       ['OFF'],
       ['NOT'],
       ['NOT']], dtype=object)

In [36]:

from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit_transform(['NOT','OFF'])
print(le.classes_)

print(y_train.shape)
print(y_test.shape)
yTrain=le.transform(y_train.flatten())
print(yTrain.shape)
print(le.classes_)


['NOT' 'OFF']
(25021, 1)
(6256, 1)
(25021,)
['NOT' 'OFF']


In [0]:
yTest=le.transform(y_test.flatten())


In [38]:
set(yTest)

{0, 1}

In [0]:
from torch.utils.data import Dataset
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=64
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytrain[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        x_train_mask_pytorch=torch.tensor(attention_mask)
        
        return x_train_pytorch,x_train_mask_pytorch,y_train_pytorch
        
        
    def __len__(self):
        return len(self.xytrain[0])

In [0]:
#from torch.utils.data import Dataset
#tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=64
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytest[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength-2]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        x_test_mask_pytorch=torch.tensor(attention_mask)
        
        return x_test_pytorch,x_test_mask_pytorch,y_test_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytest[0])

In [0]:
xytrain=[sentence_train,yTrain]
tdataset = GreekTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)

In [0]:
xytest=[sentence_test,yTest]
tedataset = GreekTestDataset(xytest)
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)

In [43]:
from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
model=bfsc.from_pretrained('bert-base-multilingual-cased',num_labels=2,output_attentions=False,output_hidden_states=False)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [44]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/TurkishData/bertturkish.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/TurkishData/bertturkish.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [0]:
params=list(model.named_parameters())

In [0]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
{
"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
"weight_decay": 0.01,
},
{"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

In [0]:
optimizer=AdamW(model.parameters(),lr=2e-5,eps=1e-8)
#############CAN CHANGE LEARNING RATE 

In [0]:
from transformers import get_linear_schedule_with_warmup

epochs=4
total_steps=len(tdataloader)*epochs

sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  print("predictionArgmax",predictionArgmax)
  print("labelsFlattend",labelsFlattend)
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
#Also based on the following tutorials
#https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [50]:
import random
import time 

def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)
      
set_seed(42,torch.cuda.device_count())
#remove later

epochs=4
lossList=[]
max_grad_norm=1.0
for e in range(0, epochs):
    print("Start Epoch Number",(e + 1))
    print("Start Training")
    
    #Amount of time taken for training
    t1 = time.time()
    tr_loss, logging_loss = 0.0, 0.0
    model.train()
    tsteps=0
    for step, batch in enumerate(tdataloader):
        if step % 50 == 0 and not step == 0:
            print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
        
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        model.zero_grad()        
        outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"], labels=inputs["labels"])

        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        tsteps+=1
        optimizer.step()
        sch.step()

    a_tr_loss = tr_loss /(tsteps)               
    lossList.append(a_tr_loss)
    
    print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
    t2=time.time()
    print("  Training one epoch time taken",t2-t1)
    print(" Validation starts here ")
   
    t1 = time.time()
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    eval_f1=0
    eval_acc=0

    for batch in tedataloader:       
        batch = tuple(t.to(device) for t in batch)        
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        with torch.no_grad():        
            outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = (inputs["labels"]).to('cpu').numpy()
        tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
        eval_f1 = eval_f1+tmpf1score
        eval_acc=eval_acc+tmpaccscore
        nb_eval_steps += 1
        #print(" TEMP F1 score: {0:.3f}".format(tmpf1score))
        #print("TEMP  Accuracy score: {0:.3f}".format(tmpaccscore))


    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
    
    t2=time.time()
    print("  Validating one epoch time taken ",t2-t1)
    
print("ALL DONE!!!")

Start Epoch Number 1
Start Training
Batch Completed  50  of  782.    Elapsed time is  35.62844443321228
Batch Completed  100  of  782.    Elapsed time is  71.37362694740295
Batch Completed  150  of  782.    Elapsed time is  107.28227853775024
Batch Completed  200  of  782.    Elapsed time is  143.26926922798157
Batch Completed  250  of  782.    Elapsed time is  179.2598295211792
Batch Completed  300  of  782.    Elapsed time is  215.2375681400299
Batch Completed  350  of  782.    Elapsed time is  251.22631001472473
Batch Completed  400  of  782.    Elapsed time is  287.20609521865845
Batch Completed  450  of  782.    Elapsed time is  323.2094359397888
Batch Completed  500  of  782.    Elapsed time is  359.21510791778564
Batch Completed  550  of  782.    Elapsed time is  395.1975119113922
Batch Completed  600  of  782.    Elapsed time is  431.09568190574646
Batch Completed  650  of  782.    Elapsed time is  466.73138880729675
Batch Completed  700  of  782.    Elapsed time is  502.350148

In [0]:
torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/TurkishData/bertturkish.pth.tar')


In [0]:
!unzip -P ****** -qq '/content/drive/My Drive/TurkishData/task_a_baseline.zip'
!unzip -P ****** -qq '/content/drive/My Drive/TurkishData/offenseval-tr-testset-v1.zip'

unzip:  cannot find or open /content/drive/My Drive/DanishData/DanishPredict.zip, /content/drive/My Drive/DanishData/DanishPredict.zip.zip or /content/drive/My Drive/DanishData/DanishPredict.zip.ZIP.
unzip:  cannot find or open /content/drive/My Drive/DanishData/Danish.zip, /content/drive/My Drive/DanishData/Danish.zip.zip or /content/drive/My Drive/DanishData/Danish.zip.ZIP.


In [0]:
ygiven=[]
ypredicted=[]

def convertToInt(val):
    if not val:
        return 0    
    try:
        return np.int64(val)
    except:        
        return np.int64(0)




In [0]:
##Use blob
#GET THE DATA FROM THE PANDAS FRAME
def readData1():
  headers=['id','tweet']
  greekDataTest = pd.read_csv("offenseval-tr-testset-v1.tsv", delimiter='\t',names=headers,
                                converters={"id":convertToInt})
  greekDataTest=greekDataTest[1:]
  print(greekDataTest.head())
  print(greekDataTest.dtypes)
  print(greekDataTest.shape)
  
  dfnumpy=greekDataTest.to_numpy();
  X=dfnumpy[:, 1].reshape(-1, 1)
  tid=dfnumpy[:, 0].reshape(-1, 1)
  print(tid)
  arrt=X[:,0]
  #allTokens=preprocess1(arrt)
  preprocessedTweets=arrt
  return preprocessedTweets,tid

In [66]:
preprocessedTweets,tid=readData1()


      id                                              tweet
1  41993    @USER Sayın başkanım bu şekilde devam inşallah👏
2  23000  Herkes gevşekliği kadar duyar kasıyor,hayat bö...
3  42478  Olgun ilişkisi olan arkadaş size en güzel hedi...
4  21748  @USER @USER Burada atıp tutacağına o kötü koşu...
5  13607   @USER İşte o onur dediğin sende yok sorun o işte
id        int64
tweet    object
dtype: object
(3515, 2)
[[41993]
 [23000]
 [42478]
 ...
 [45705]
 [29225]
 [15130]]


In [0]:
from torch.utils.data import Dataset
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekPredictDataset(Dataset):
    def __init__(self,xypredict):
        self.xypredict = xypredict
        self.maxlength=64
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xypredict[0][index]))
        if len(tokenized_review) > self.maxlength:
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_predict_pytorch = torch.tensor(ids_of_sentence_word)
        x_predict_mask_pytorch=torch.tensor(attention_mask)
        tid_predict_pytorch=torch.tensor(self.xypredict[1][index])
        
        return x_predict_pytorch,x_predict_mask_pytorch,tid_predict_pytorch
       
    def __len__(self):
        return len(self.xypredict[0])
 


In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
def calculateScore(predictions):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  yres.append(predictionArgmax.flatten())
  print("predictionArgmax",predictionArgmax)
 

In [0]:
yres=[]
finalTid=[]
def predictingData(pTweets,tid):
  #https://colab.research.google.com/drive/1Y4o3jh3ZH70tl6mCd76vz_IxX23biCPP#scrollTo=1M296yz577fV
  ids_of_sentence=[]
  predictedLabels,trueLabels=[],[]
  
  map_location=""
  xypredict=[pTweets,tid.flatten()]
  
  tdataset = GreekPredictDataset(xypredict)
  tsampler=RandomSampler(tdataset)
  predictdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
  print(device.type)
  '''model=bfsc.from_pretrained('bert-base-multilingual-cased',num_labels=2,output_attentions=False,output_hidden_states=False)
  if device.type=="cpu":
    model.to(device)
    map_location='cpu'
  else:
    model.cuda()
    map_location=lambda storage, loc: storage.cuda()
  params=list(model.named_parameters())
  eval_f1=0
  eval_acc=0
  nb_eval_steps=0
  checkpoint = torch.load('/content/drive/My Drive/TurkishData/bertturkish.pth.tar',map_location=map_location)
  print("Hello")
  model.load_state_dict(checkpoint['state_dict'])'''
  model.eval()
  i=0
  for batch in predictdataloader:
      print(i)
      i=i+1
      batch = tuple(t.to(device) for t in batch)        
      inputs = {"input_ids": batch[0], "attention_mask": batch[1], "tids":batch[2]}
      
      with torch.no_grad():       
          outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      predictedLabels.append(logits)
      tidl=(inputs["tids"]).to('cpu').numpy()
      finalTid.append(tidl)
      calculateScore(logits)
      
  return predictedLabels,finalTid

In [72]:
preprocessedTweets,tid=readData1()

      id                                              tweet
1  41993    @USER Sayın başkanım bu şekilde devam inşallah👏
2  23000  Herkes gevşekliği kadar duyar kasıyor,hayat bö...
3  42478  Olgun ilişkisi olan arkadaş size en güzel hedi...
4  21748  @USER @USER Burada atıp tutacağına o kötü koşu...
5  13607   @USER İşte o onur dediğin sende yok sorun o işte
id        int64
tweet    object
dtype: object
(3515, 2)
[[41993]
 [23000]
 [42478]
 ...
 [45705]
 [29225]
 [15130]]


In [73]:
predictedLabels,finalTid=predictingData(preprocessedTweets,tid)

cuda
0
predictionArgmax [0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0]
1
predictionArgmax [0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0]
2
predictionArgmax [0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0]
3
predictionArgmax [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1]
4
predictionArgmax [1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0]
5
predictionArgmax [1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
6
predictionArgmax [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0]
7
predictionArgmax [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0]
8
predictionArgmax [1 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1]
9
predictionArgmax [1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0]
10
predictionArgmax [1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0]
11
predictionArgmax [0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 

In [0]:
yans=[yres[i].flatten().tolist() for i in range(len(yres))]
ytid=[finalTid[i].flatten().tolist() for i in range(len(finalTid))]

In [0]:
from itertools import chain

yans=list(chain.from_iterable(yans))
ytid=list(chain.from_iterable(ytid))

In [76]:
len(ytid)

3515

In [0]:
yans1=["NOT" if yans[i]==0 else "OFF" for i in range(len(yans))]

In [78]:
print(yans1.count("OFF"))
print(yans1.count("NOT"))

666
2849


In [80]:
#z={'tweet':sentence_predict[],'subtask_a':y_predict}
#print(len(z))
C = {'id': ytid,
        'predicted': yans1,
    }
df = pd.DataFrame(C)
print (df[df['id'] == 400] )

export_csv = df.to_csv ('/content/drive/My Drive/TurkishData/TurkishAnswer.csv', index = None, header=False)

Empty DataFrame
Columns: [id, predicted]
Index: []


from 80 ,10 ,10 split getting  # F1 score: 0.836 Accuracy score: 0.867